In [76]:
import csv
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('./data') if isfile(join('./data', f))]

In [77]:
import re
import json
from collections import defaultdict

In [102]:
text = list()
labels = list()
data_source = defaultdict(int)
total_count = 12322
not_count = 0
for file in onlyfiles:        
    with open('data/' + file, newline='', encoding="utf8") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        if 'trump' not in file.lower():
            label = 'NoTrump'
            if not_count == total_count:
                continue
        else:
            label = 'Trump'        
        text_ind = 1
        if "MORE_TRUMP.csv" in file:
            text_ind = 0
            source = 'realDonaldTrump'
        else:
            source = file[:-11]        
        for row in csv_reader:            
            if line_count != 0:
                line = re.sub('[\s\n\t#@]+', ' ', row[text_ind])
                line = re.sub(r'http\S+', '', line).strip()            
                if len(line) == 0:
                    continue
                text.append(line)                
                labels.append(label)            
                data_source[source] += 1
                if label != 'Trump':
                    not_count += 1
                    if not_count == total_count:
                        break                
            line_count += 1
                

In [103]:
data_source

defaultdict(int,
            {'realDonaldTrump': 12322,
             'SecretaryCarson': 1242,
             'MartinOMalley': 2211,
             'JebBush': 2757,
             'BarackObama': 2880,
             'HillaryClinton': 1774,
             'BernieSanders': 1458})

In [104]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = \
train_test_split(text, labels, test_size=0.13, random_state=42)
with open("train.csv", "w", newline='', encoding='utf8') as csv_file:
    csv_writer = csv.writer(csv_file)
    for i in range(len(train_data)):
        csv_writer.writerow([train_data[i], train_labels[i]])
with open("dev.csv", "w", newline='', encoding='utf8') as csv_file:
    csv_writer = csv.writer(csv_file)
    for i in range(len(test_data)):
        csv_writer.writerow([test_data[i], test_labels[i]])

In [105]:
len(train_data[0])

113

In [106]:
from collections import Counter
c = Counter(train_labels)
c

Counter({'NoTrump': 10681, 'Trump': 10759})

In [84]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=3, max_df=200)
trainX = vectorizer.fit_transform(train_data)
testX = vectorizer.transform(test_data)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_labels)
trainy = le.transform(train_labels)
testy = le.transform(test_labels)

In [85]:
from sklearn.linear_model import LogisticRegression
cls = LogisticRegression(random_state=0, solver='lbfgs', max_iter=10000)
cls.fit(trainX, trainy)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False)

In [86]:
def evaluate(X, yt, cls, name='data'):
	"""Evaluated a classifier on the given labeled data using accuracy."""
	from sklearn import metrics
	yp = cls.predict(X)
	acc = metrics.accuracy_score(yt, yp)
	print("  Accuracy on %s  is: %s" % (name, acc))

In [87]:
evaluate(testX, testy, cls)

  Accuracy on data  is: 0.9107088063823259
